In [3]:
import numpy as np 
import re

In [4]:
L = 32 
RA = 4
RB = 2
RC = 4
PA = 2
PB = 2
#PC = 1 so fuck it we guuci 

edge_re = re.compile(
    r"\((\d+),(\d+),(\d+)\)x"
    r"\((\d+),(\d+),(\d+)\)\s*->\s*"
    r"\((\d+),(\d+)\)"
)

def parse_edges(filename):
    edges = []

    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            m = edge_re.match(line)
            if not m:
                raise ValueError(f"Could not parse line:\n{line}")

            la, ra, pa, lb, rb, pb, lc, rc = map(int, m.groups())

            A = (la, ra, pa)
            B = (lb, rb, pb)
            C = (lc, rc)

            edges.append((A, B, C))

    return edges


In [5]:
edges = parse_edges("probe_data.txt")

In [6]:
edges
#are the triples of a_coord, b_coord that hit lane x in the c_coordinate, 
#always also have lane x? that is is data redistributed among lanes after the 
#tensor core does the mma? 


[((0, 0, 0), (0, 0, 0), (0, 0)),
 ((0, 0, 0), (4, 0, 0), (0, 1)),
 ((0, 0, 0), (8, 0, 0), (1, 0)),
 ((0, 0, 0), (12, 0, 0), (1, 1)),
 ((0, 0, 0), (16, 0, 0), (2, 0)),
 ((0, 0, 0), (20, 0, 0), (2, 1)),
 ((0, 0, 0), (24, 0, 0), (3, 0)),
 ((0, 0, 0), (28, 0, 0), (3, 1)),
 ((0, 0, 1), (0, 0, 1), (0, 0)),
 ((0, 0, 1), (4, 0, 1), (0, 1)),
 ((0, 0, 1), (8, 0, 1), (1, 0)),
 ((0, 0, 1), (12, 0, 1), (1, 1)),
 ((0, 0, 1), (16, 0, 1), (2, 0)),
 ((0, 0, 1), (20, 0, 1), (2, 1)),
 ((0, 0, 1), (24, 0, 1), (3, 0)),
 ((0, 0, 1), (28, 0, 1), (3, 1)),
 ((0, 1, 0), (0, 0, 0), (0, 2)),
 ((0, 1, 0), (4, 0, 0), (0, 3)),
 ((0, 1, 0), (8, 0, 0), (1, 2)),
 ((0, 1, 0), (12, 0, 0), (1, 3)),
 ((0, 1, 0), (16, 0, 0), (2, 2)),
 ((0, 1, 0), (20, 0, 0), (2, 3)),
 ((0, 1, 0), (24, 0, 0), (3, 2)),
 ((0, 1, 0), (28, 0, 0), (3, 3)),
 ((0, 1, 1), (0, 0, 1), (0, 2)),
 ((0, 1, 1), (4, 0, 1), (0, 3)),
 ((0, 1, 1), (8, 0, 1), (1, 2)),
 ((0, 1, 1), (12, 0, 1), (1, 3)),
 ((0, 1, 1), (16, 0, 1), (2, 2)),
 ((0, 1, 1), (20, 0, 1), (

In [7]:
def get_A_B_coords_by_C_lane(edges): 
  group_by_c_lane = {i:[] for i in range(32)}
  for e in edges: 
    a_coord,b_coord, c_coord = e
    group_by_c_lane[c_coord[0]].append(e)
  return group_by_c_lane  

In [8]:
group_by_c_lane_dict = get_A_B_coords_by_C_lane(edges)

In [9]:
def get_C_inv_map(edges):
    C_inv_map = {c_coord: [] for _, _, c_coord in edges}

    for a, b, c in edges:
        C_inv_map[c].append((a, b))

    # sort each list lexicographically
    for c in C_inv_map:
        C_inv_map[c].sort()

    return C_inv_map
  

In [10]:
C_inv_map = get_C_inv_map(edges)

In [11]:
print(C_inv_map[0,0])
print(C_inv_map[0,1])
print(C_inv_map[0,2])
print(C_inv_map[0,3])

[((0, 0, 0), (0, 0, 0)), ((0, 0, 1), (0, 0, 1)), ((0, 2, 0), (0, 1, 0)), ((0, 2, 1), (0, 1, 1)), ((1, 0, 0), (1, 0, 0)), ((1, 0, 1), (1, 0, 1)), ((1, 2, 0), (1, 1, 0)), ((1, 2, 1), (1, 1, 1)), ((2, 0, 0), (2, 0, 0)), ((2, 0, 1), (2, 0, 1)), ((2, 2, 0), (2, 1, 0)), ((2, 2, 1), (2, 1, 1)), ((3, 0, 0), (3, 0, 0)), ((3, 0, 1), (3, 0, 1)), ((3, 2, 0), (3, 1, 0)), ((3, 2, 1), (3, 1, 1))]
[((0, 0, 0), (4, 0, 0)), ((0, 0, 1), (4, 0, 1)), ((0, 2, 0), (4, 1, 0)), ((0, 2, 1), (4, 1, 1)), ((1, 0, 0), (5, 0, 0)), ((1, 0, 1), (5, 0, 1)), ((1, 2, 0), (5, 1, 0)), ((1, 2, 1), (5, 1, 1)), ((2, 0, 0), (6, 0, 0)), ((2, 0, 1), (6, 0, 1)), ((2, 2, 0), (6, 1, 0)), ((2, 2, 1), (6, 1, 1)), ((3, 0, 0), (7, 0, 0)), ((3, 0, 1), (7, 0, 1)), ((3, 2, 0), (7, 1, 0)), ((3, 2, 1), (7, 1, 1))]
[((0, 1, 0), (0, 0, 0)), ((0, 1, 1), (0, 0, 1)), ((0, 3, 0), (0, 1, 0)), ((0, 3, 1), (0, 1, 1)), ((1, 1, 0), (1, 0, 0)), ((1, 1, 1), (1, 0, 1)), ((1, 3, 0), (1, 1, 0)), ((1, 3, 1), (1, 1, 1)), ((2, 1, 0), (2, 0, 0)), ((2, 1, 1), (

In [12]:
def T_lb_plus_4(a, b):
    la, ra, pa = a
    lb, rb, pb = b
    return (la, ra, pa), (lb + 4, rb, pb)

def T_ra_plus_1(a, b):
    la, ra, pa = a
    lb, rb, pb = b
    return (la, ra + 1, pa), (lb, rb, pb)

def T_both(a, b):
    la, ra, pa = a
    lb, rb, pb = b
    return (la, ra + 1, pa), (lb + 4, rb, pb)
  
  
def normalize(pairs):
    return set(pairs)

def test_rc_invariant(C_inv_map, lc=0):
    base = normalize(C_inv_map[(lc, 0)])

    tests = {
        1: lambda p: T_lb_plus_4(*p),
        2: lambda p: T_ra_plus_1(*p),
        3: lambda p: T_both(*p),
    }

    for rc, T in tests.items():
        transformed = set(T((a,b)) for (a,b) in base)
        target = normalize(C_inv_map[(lc, rc)])

        if transformed != target:
            print(f"❌ invariant failed for rc = {rc}")
            print("missing:", transformed - target)
            print("extra:", target - transformed)
            return False

    print("✅ rc-invariant holds for lc =", lc)
    return True



In [13]:
for i in range(32):
  test_rc_invariant(C_inv_map, lc=i)

✅ rc-invariant holds for lc = 0
✅ rc-invariant holds for lc = 1
✅ rc-invariant holds for lc = 2
✅ rc-invariant holds for lc = 3
✅ rc-invariant holds for lc = 4
✅ rc-invariant holds for lc = 5
✅ rc-invariant holds for lc = 6
✅ rc-invariant holds for lc = 7
✅ rc-invariant holds for lc = 8
✅ rc-invariant holds for lc = 9
✅ rc-invariant holds for lc = 10
✅ rc-invariant holds for lc = 11
✅ rc-invariant holds for lc = 12
✅ rc-invariant holds for lc = 13
✅ rc-invariant holds for lc = 14
✅ rc-invariant holds for lc = 15
✅ rc-invariant holds for lc = 16
✅ rc-invariant holds for lc = 17
✅ rc-invariant holds for lc = 18
✅ rc-invariant holds for lc = 19
✅ rc-invariant holds for lc = 20
✅ rc-invariant holds for lc = 21
✅ rc-invariant holds for lc = 22
✅ rc-invariant holds for lc = 23
✅ rc-invariant holds for lc = 24
✅ rc-invariant holds for lc = 25
✅ rc-invariant holds for lc = 26
✅ rc-invariant holds for lc = 27
✅ rc-invariant holds for lc = 28
✅ rc-invariant holds for lc = 29
✅ rc-invariant holds

In [14]:
def get_AxB_to_C(edges): 
  forward_rel = {(a,b):[] for a,b,_ in edges}
  for a,b,c in edges: 
    forward_rel[(a,b)].append(c)
    
  return forward_rel
  
    

In [15]:
forward_rel = get_AxB_to_C(edges)

In [16]:
for la in range(L):
  for lb in range(L): 
    for ra in range(RA): 
      for rb in range(RB):
        for pa in range(PA): 
          for pb in range(PB): 
            x = ((la,ra,pa),(lb,rb,pb))
            if x in forward_rel: 
              image_x = forward_rel[x]
              #print(len(image_x)==1)

In [17]:
C_inv_map[0,0]

[((0, 0, 0), (0, 0, 0)),
 ((0, 0, 1), (0, 0, 1)),
 ((0, 2, 0), (0, 1, 0)),
 ((0, 2, 1), (0, 1, 1)),
 ((1, 0, 0), (1, 0, 0)),
 ((1, 0, 1), (1, 0, 1)),
 ((1, 2, 0), (1, 1, 0)),
 ((1, 2, 1), (1, 1, 1)),
 ((2, 0, 0), (2, 0, 0)),
 ((2, 0, 1), (2, 0, 1)),
 ((2, 2, 0), (2, 1, 0)),
 ((2, 2, 1), (2, 1, 1)),
 ((3, 0, 0), (3, 0, 0)),
 ((3, 0, 1), (3, 0, 1)),
 ((3, 2, 0), (3, 1, 0)),
 ((3, 2, 1), (3, 1, 1))]

In [18]:
for x,y in zip(C_inv_map[3,1], C_inv_map[4,1]):
  print(x[1] == y[1])
  print(x[1],y[1])

  

False
(28, 0, 0) (4, 0, 0)
False
(28, 0, 1) (4, 0, 1)
False
(28, 1, 0) (4, 1, 0)
False
(28, 1, 1) (4, 1, 1)
False
(29, 0, 0) (5, 0, 0)
False
(29, 0, 1) (5, 0, 1)
False
(29, 1, 0) (5, 1, 0)
False
(29, 1, 1) (5, 1, 1)
False
(30, 0, 0) (6, 0, 0)
False
(30, 0, 1) (6, 0, 1)
False
(30, 1, 0) (6, 1, 0)
False
(30, 1, 1) (6, 1, 1)
False
(31, 0, 0) (7, 0, 0)
False
(31, 0, 1) (7, 0, 1)
False
(31, 1, 0) (7, 1, 0)
False
(31, 1, 1) (7, 1, 1)


In [19]:
def transform_base_to(lc, rc, a, b):
    la, ra, pa = a
    lb, rb, pb = b

    a2 = (
        la + 4 * (lc // 4),
        ra + (rc // 2),
        pa
    )

    b2 = (
        lb + 4 * (rc % 2) + 8 * (lc % 4),
        rb,
        pb
    )

    return a2, b2

def transform_to_base(lc, rc, a, b):
    la, ra, pa = a
    lb, rb, pb = b

    a0 = (
        la - 4 * (lc // 4),
        ra - (rc // 2),
        pa
    )

    b0 = (
        lb - 4 * (rc % 2) - 8 * (lc % 4),
        rb,
        pb
    )

    return a0, b0

edge_set = set(edges)

base_edges = set(
    (a, b)
    for (a, b, c) in edges
    if c == (0, 0)
)

def check_forward(edges):
    edge_set = set(edges)

    for a0, b0 in base_edges:
        for lc in range(L):
            for rc in range(4):
                a2, b2 = transform_base_to(lc, rc, a0, b0)

                if (a2, b2, (lc, rc)) not in edge_set:
                    print("❌ forward failed")
                    print("base:", a0, b0)
                    print("target:", a2, b2, (lc, rc))
                    return False

    print("✅ forward transform holds")
    return True
  
def check_backward(edges):
    edge_set = set(edges)

    for (a, b, (lc, rc)) in edges:
        a0, b0 = transform_to_base(lc, rc, a, b)

        if (a0, b0) not in base_edges:
            print("❌ backward failed")
            print("edge:", a, b, (lc, rc))
            print("maps to base:", a0, b0)
            return False

    print("✅ backward transform holds")
    return True



In [20]:
check_forward(edges)
check_backward(edges)


✅ forward transform holds
✅ backward transform holds


True

In [21]:
C_inv_map[0,0]

[((0, 0, 0), (0, 0, 0)),
 ((0, 0, 1), (0, 0, 1)),
 ((0, 2, 0), (0, 1, 0)),
 ((0, 2, 1), (0, 1, 1)),
 ((1, 0, 0), (1, 0, 0)),
 ((1, 0, 1), (1, 0, 1)),
 ((1, 2, 0), (1, 1, 0)),
 ((1, 2, 1), (1, 1, 1)),
 ((2, 0, 0), (2, 0, 0)),
 ((2, 0, 1), (2, 0, 1)),
 ((2, 2, 0), (2, 1, 0)),
 ((2, 2, 1), (2, 1, 1)),
 ((3, 0, 0), (3, 0, 0)),
 ((3, 0, 1), (3, 0, 1)),
 ((3, 2, 0), (3, 1, 0)),
 ((3, 2, 1), (3, 1, 1))]

In [22]:
def C_inv_0_0_generator(i):
  assert 0 <= i < 16
  
  la,lb = i // 4, i // 4
  ra,rb = 2*((i//2)%2),((i//2)%2)
  pa,pb = i % 2, i % 2
  return ((la,ra,pa),(lb,rb,pb)) 

In [23]:
def generate_C_inv_0_0():
  gen = []
  for i in range(16):
    gen.append(C_inv_0_0_generator(i))
    
  return set(gen)

In [24]:
print(set(C_inv_map[0,0]) == generate_C_inv_0_0())

True


In [25]:
#finding the predicate: 
for a,b,c in edges: 
  print(a[2] == b[2])
  print(a[1] == 2*b[1])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
Tru

In [26]:
def generate_inv_map_by_k(k,lc,rc):
  la = (k//4) + (4*(lc//4))
  lb = (k//4) + (8*(lc % 4)) + (4*(rc % 2))
  ra = (2*((k//2)%2)) + ((rc//2))
  rb = (k//2)%2
  pa = k % 2
  pb = k % 2
  return((la,ra,pa),(lb,rb,pb))
  
def generate_inv_map(lc,rc): 
  L = []
  for k in range(16):
    L.append(generate_inv_map_by_k(k,lc,rc))
    
  return set(L)

def verify(C_inv_map): 
  for lc in range(L): 
    for rc in range(RC): 
      inv_map = set(C_inv_map[lc,rc])
      generated_inv_map = generate_inv_map(lc,rc)
      if inv_map != generated_inv_map:
        return False
      
  return True

In [27]:
verify(C_inv_map)

True

In [28]:
import numpy as np
import re

def parse_mma_dump(filename):
    """
    Parses kernel printf dump of form:

    lane  0 | A: [ a a | a a | a a | a a ]
              B: [ b b | b b ]
              C: [ c c c c ]

    Returns:
        A: (32, 4, 2)
        B: (32, 2, 2)
        C: (32, 4)
    """

    A = np.zeros((32, 4, 2), dtype=np.float32)
    B = np.zeros((32, 2, 2), dtype=np.float32)
    C = np.zeros((32, 4), dtype=np.float32)

    lane_re = re.compile(r"lane\s+(\d+)")
    float_re = re.compile(r"[-+]?\d*\.\d+|\d+")

    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # lane id
            m = lane_re.search(line)
            if not m:
                continue

            lane = int(m.group(1))

            # extract all floats in order
            vals = [float(x) for x in float_re.findall(line)]

            # Expected order:
            # A: 8 floats
            # B: 4 floats
            # C: 4 floats
            #
            # total = 16
            vals = vals[1:]
            if len(vals) != 16:
                raise ValueError(
                    f"Lane {lane}: expected 16 floats, got {len(vals)}"
                )

            # A
            A[lane, 0, 0] = vals[0]
            A[lane, 0, 1] = vals[1]

            A[lane, 1, 0] = vals[2]
            A[lane, 1, 1] = vals[3]

            A[lane, 2, 0] = vals[4]
            A[lane, 2, 1] = vals[5]

            A[lane, 3, 0] = vals[6]
            A[lane, 3, 1] = vals[7]

            # B
            B[lane, 0, 0] = vals[8]
            B[lane, 0, 1] = vals[9]

            B[lane, 1, 0] = vals[10]
            B[lane, 1, 1] = vals[11]

            # C
            C[lane, 0] = vals[12]
            C[lane, 1] = vals[13]
            C[lane, 2] = vals[14]
            C[lane, 3] = vals[15]

    return A, B, C


In [29]:
A,B,C = parse_mma_dump("data.txt")
print(A.shape)
print(B.shape)
print(C.shape)

(32, 4, 2)
(32, 2, 2)
(32, 4)


In [30]:
def do_mma(A,B,C): 
  C_out = np.zeros_like(C) 
  for lc in range(L): 
    for rc in range(RC):
      pre_image = generate_inv_map(lc,rc)
      for a,b in pre_image: 
        C_out[lc,rc] += A[a]*B[b]
        
  return C_out

In [31]:
C_out = do_mma(A,B,C)

In [32]:
np.allclose(C,C_out,atol=1e-3)

True

In [33]:
C_out

array([[  4.534694 ,  11.097038 ,   3.7910106,   9.331585 ],
       [ 17.657974 ,  24.219873 ,  14.870822 ,  20.411024 ],
       [ 30.778736 ,  37.34064  ,  25.969025 ,  31.509226 ],
       [ 43.90254  ,  50.464436 ,  37.049423 ,  42.589626 ],
       [ 12.534492 ,  35.097748 ,  10.388456 ,  29.210857 ],
       [ 57.659412 ,  80.22182  ,  48.032078 ,  66.85428  ],
       [102.78079  , 125.3432   ,  85.73678  , 104.55898  ],
       [147.90561  , 170.468    , 123.38118  , 142.20337  ],
       [ 20.533653 ,  59.097504 ,  16.985792 ,  49.08711  ],
       [ 97.65893  , 136.2213   ,  81.188126 , 113.28962  ],
       [174.78079  , 213.3432   , 145.49387  , 177.59537  ],
       [251.90561  , 290.46802  , 209.69685  , 241.79836  ],
       [ 28.533152 ,  83.0975   ,  23.585133 ,  68.96747  ],
       [137.65894  , 192.22133  , 114.34968  , 159.73239  ],
       [246.78079  , 301.34317  , 205.26025  , 250.64293  ],
       [355.90564  , 410.468    , 296.0256   , 341.40836  ],
       [ 36.56482  , 107

In [34]:
C

array([[  4.5347,  11.0972,   3.791 ,   9.3317],
       [ 17.6583,  24.2203,  14.871 ,  20.4113],
       [ 30.7792,  37.3412,  25.9694,  31.5097],
       [ 43.9033,  50.4653,  37.05  ,  42.5902],
       [ 12.5346,  35.0979,  10.3884,  29.2108],
       [ 57.6597,  80.2222,  48.0321,  66.8544],
       [102.7812, 125.3438,  85.7371, 104.5593],
       [147.9062, 170.4688, 123.3816, 142.2039],
       [ 20.5337,  59.0977,  16.9856,  49.087 ],
       [ 97.6592, 136.2217,  81.1881, 113.2897],
       [174.7812, 213.3438, 145.4941, 177.5957],
       [251.9062, 290.4688, 209.6973, 241.7988],
       [ 28.5332,  83.0977,  23.5849,  68.9674],
       [137.6592, 192.2217, 114.3497, 159.7325],
       [246.7812, 301.3438, 205.2607, 250.6436],
       [355.9062, 410.4688, 296.0264, 341.4092],
       [ 36.565 , 107.1924,  30.1523,  88.7535],
       [177.8164, 248.4414, 147.3541, 205.9557],
       [319.0625, 389.6875, 264.7451, 323.3467],
       [460.3125, 530.9375, 381.9482, 440.5498],
       [ 44.5645, 13

In [43]:
def predicate(A_B_pair):
    la, ra, pa = A_B_pair[0]
    lb, rb, pb = A_B_pair[1]

    if pa != pb:
        return False, -1, -1

    k0 = pa
    k1 = rb
    k_low = k0 + 2*k1

    if la % 4 != lb % 4:
        return False, -1, -1
    k_high = la % 4
    k = k_low + 4*k_high
    if not (0 <= k <= 15):
        return False, -1, -1

    rc1 = ra - 2*rb
    if rc1 not in (0, 1):
        return False, -1, -1

    # ---- divisibility guards ----
    if (lb - k_high) % 4 != 0:
        return False, -1, -1

    rc0 = ((lb - k_high) // 4) % 2

    if (lb - k_high - 4*rc0) % 8 != 0:
        return False, -1, -1

    lc_low = (lb - k_high - 4*rc0) // 8

    if (la - k_high) % 4 != 0:
        return False, -1, -1

    lc_high = (la - k_high) // 4

    if not (0 <= lc_low <= 3 and 0 <= lc_high <= 7):
        return False, -1, -1

    lc = lc_low + 4*lc_high
    rc = rc0 + 2*rc1

    return True, lc, rc


  
  

def forward(A_B_pair): 
  Truth,lc,rc = predicate(A_B_pair) 
  if Truth:
    return (lc,rc)
  else:
    return None
  

In [44]:
edge_set = set(edges)
constructed_edge_set = []
for la in range(L):
  for ra in range(RA): 
    for pa in range(PA): 
      for lb in range(L): 
        for rb in range(RB): 
          for pc in range(PB): 
            a = (la,ra,pa)
            b = (lb,rb,pb)
            A_B_pair = (a,b)
            out = forward(A_B_pair)
            if out: 
              constructed_edge_set.append((a,b,out))
              
              
print(set(constructed_edge_set)==edge_set)

False


In [47]:
print(edge_set.pop())
print(constructed_edge_set.pop())

((9, 3, 1), (25, 1, 1), (11, 2))
((31, 3, 1), (31, 1, 1), (31, 3))
